In [1]:
import pandas as pd
import pickle 
import os
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
from finenv.save_model import *
import psutil
import ray
from datetime import datetime
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility
from ray.rllib.agents import ppo
from ray.rllib.algorithms.td3 import TD3Config
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig

In [2]:
csv =  True
if csv:
    train = pd.read_csv('dataset/train_data.csv')
    train = train.set_index(train.columns[0])
    #train.reset_index(drop=True, inplace=True)
    train.index.names = ['']
    INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

else: 
    train = pd.read_csv('dataset/idxetf_train.csv')
    train = train.set_index(train.columns[0])
    train.index.names = ['']
    train = train.sort_values(['date'])
    train.index = train['date'].factorize()[0]
    INDICATORS = ['macd','rsi','cci','adx']
    train.head()
    
stock_dimension = len(train.tic.unique())
buy_cost_list = sell_cost_list = [0.001] * stock_dimension    
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
train.shape

Stock Dimension: 82, State Space: 821


(193028, 18)

In [3]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))


In [4]:
ray.shutdown()
register_env("finrl", env_creator)
#ray.init(num_cpus=122,dashboard_port=8080)
print(f"ray is being initialized")

ray is being initialized


In [5]:
config = TD3Config().training(lr=0.01).resources(num_gpus=1).framework(framework="torch").rollouts(num_rollout_workers=10)
config["model"]["fcnet_hiddens"] = [1024,1024]
config['train_batch_size'] = 1024
#print(config.to_dict())  
trainer = config.build(env="finrl") 

2023-04-11 08:04:01,299	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(pid=579) Inconsistency detected by ld.so: ../elf/dl-tls.c: 517: _dl_allocate_tls_init: Assertion `listp != NULL' failed!
2023-04-11 08:04:06,624	WARNING worker.py:1866 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff2e829caa24f1009103ec4b2801000000 Worker ID: 04fe655d6f002d164e4656247e37d32f125826212295e0fe7ce8bb7a Node ID: 2ba7a491a8a3d86f5178c23e7d7f21f2a199f50b1a7c22dd5329ec61 Worker IP address: 172.21.2.233 Worker port: 43101 Worker PID: 579 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed un

In [6]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes = 24
agent_name = 'td3'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
date = datetime.now().strftime('%y%m%d')

while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    rwd = results[-1]['episode_reward_mean']
    if ep % 1 == 0:
        print(f'Mean Rwd:{rwd}')
    if ep % 100 == 0:
        #cwd_checkpoint = "results/checkpoints/" +  + '_' + str(ep)
        cwd_checkpoint = f"model/{agent_name}_{date}"
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint{ep} saved in directory {cwd_checkpoint}")
bar.close()

Episodes:   0%|          | 0/24 [00:00<?, ?it/s]

2023-04-11 08:04:49,484	WARNING replay_buffer.py:61 -- Estimated max memory usage for replay buffer is 6.946 GB (1000000.0 batches of size 1, 6946 bytes each), available system memory is 25.214959616 GB


Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
Mean Rwd:nan
(RolloutWorker pid=592) day: 2353, episode: 3
(RolloutWorker pid=592) begin_total_asset: 1000000.00
(RolloutWorker pid=592) end_total_asset: 6219271.96
(RolloutWorker pid=592) total_reward: 5219271.96
(RolloutWorker pid=592) total_cost: 249172.59
(RolloutWorker pid=592) total_trades: 148655
(RolloutWorker pid=592) Sharpe: 1.143
(RolloutWorker pid=592) =================================
(RolloutWorker pid=605) day: 2353, episode: 3
(RolloutWorker pid=605) begin_total_asset: 1000000.00
(RolloutWorker pid=605) end_total_asset: 6639588.88
(RolloutWorker pid=605) total_reward: 5639588.88
(RolloutWorker pid=605) total_cost: 249495.30
(RolloutWorker pid=605) total_trades: 148867
(RolloutWorker pid=60

In [30]:
ep_rw = np.array(results[-1]['hist_stats']['episode_reward']).mean()
inf_t = results[-1]['sampler_perf']['mean_inference_ms']
total_ts = results[-1]['timesteps_total']
print(f'EP Rwd:{ep_rw:.3f} Avg step time:{inf_t:.3f} Total ts:{total_ts}')


EP Rwd:52749.827 Avg step time:2.279 Total ts:25000


In [ ]:
#Save latest ckpt point
cwd_checkpoint = f"model/org_{agent_name}_{date}_{ep}"
trainer.save(cwd_checkpoint)
#Extract model weights 
model_weights = trainer.get_policy().get_weights()
print('passed model weights')
config2 = TD3Config()
print('config created')
config2 = config2.environment(env_config={'hmax':500,'initial_amount':1000000})  
config2 = config2.rollouts(num_rollout_workers=0) 
config2 = config2.framework(framework="torch")
config2["model"]["fcnet_hiddens"] = [1024,1024]
trainer2 = config2.build(env="finrl") 
trainer2.get_policy().set_weights(model_weights)
print('New Weights loaded. ')
ckpt2 = f"{cwd_checkpoint}_wt"
trainer2.save(ckpt2)
zipped_filename = f'ckpt_wt{date}_{ep}.zip'


In [ ]:
zipped = zipfilem(ckpt2,zipped_filename)
st = sendfile(zipped)
print(f'file: {zipped} ; {st}')